# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

In [3]:
import csv

with open('data/open_pubs.csv') as fp:
    reader = csv.reader(fp)
    header = next(reader)
    
    for row in reader:
        print(row)
        break

['24', 'Anchor Inn', 'Upper Street, Stratford St Mary, COLCHESTER, Essex', 'CO7 6LW', '604748', '234405', '51.97039', '0.979328', 'Babergh']


In [8]:
header

['id', 'tag']

2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [8]:
import numpy as np

A = np.random.randint(0, 100, size=(10000, 10000))
B = np.random.randint(0, 100, size=(10000, 10000))

In [18]:
#np.save('A.npy', A)
np.savez('AB.npz', alexandr = A, artem = B)

In [19]:
r = np.load('AB.npz')
r

In [20]:
r.files

['alexandr', 'artem']

In [21]:
r['alexandr']

array([[ 9, 18, 53, ..., 50, 22, 59],
       [34, 30, 41, ..., 63, 23, 60],
       [10, 86,  9, ..., 92, 96, 88],
       ...,
       [90, 30, 25, ..., 65, 32,  0],
       [14, 69, 53, ...,  0, 61, 38],
       [49, 72, 60, ..., 18, 62, 29]])

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

In [13]:
import h5py

In [14]:
with h5py.File('test.h5', 'w') as hdf:
    ds1 = hdf. create_dataset('arrA', data = A)
    ds1.attrs['description'] = 'В этом дотасете хранится массив A'
    
    ds2 = hdf. create_dataset('arrB', data = B)
    ds2.attrs['description'] = 'В этом дотасете хранится массив B'
    
    

In [16]:
with h5py.File('test.h5', 'r') as hdf:
    ds1 = hdf['arrA']
    print(type(ds1))
    arr = ds1[:1000]

<class 'h5py._hl.dataset.Dataset'>


In [17]:
arr

array([[ 9, 18, 53, ..., 50, 22, 59],
       [34, 30, 41, ..., 63, 23, 60],
       [10, 86,  9, ..., 92, 96, 88],
       ...,
       [25, 15,  6, ..., 94, 78, 95],
       [16, 68, 72, ..., 15, 48, 91],
       [ 7, 50,  4, ..., 72, 44, 56]])

## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [271]:
import csv
dict_tags = {}
ids = []
with open('data/tags_sample.csv') as fp:
    reader = csv.reader(fp)
    header = next(reader)
    for row in reader:
        dict_tags.setdefault(row[0],[]).append(row[1])
dict_tags['44123']

['weeknight',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'soups-stews',
 'beans',
 'poultry',
 'american',
 'chicken',
 'stove-top',
 'dietary',
 'gluten-free',
 'comfort-food',
 'californian',
 'black-beans',
 'free-of-something',
 'meat',
 'taste-mood',
 'equipment',
 'grilling',
 '4-hours-or-less']

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [45]:
import pandas as pd
recipes = pd.read_csv('data/recipes_sample_with_filled_nsteps.csv',sep = ',',parse_dates=['submitted'])
dict_tags_str = {}
len_tags = []
for i in dict_tags:
    dict_tags_str[i] = ';'.join(dict_tags[i])
    len_tags.append(len(dict_tags[i]))
df_tags = pd.DataFrame.from_dict(dict_tags_str,orient = 'index').reset_index().rename(columns={0:'tags','index':'id'})
df_tags['n_tags'] = len_tags
recipes = recipes.combine_first(df_tags)
recipes.head(5)

,contributor_id,description,id,minutes,n_ingredients,n_steps,n_tags,name,submitted,tags
0,35193,an original recipe created by chef scott meska...,44123,90,18.0,11,25.0,george s at the cove black bean soup,2002-10-25,weeknight;time-to-make;course;main-ingredient;...
1,91970,my children and their friends ask for my homem...,67664,10,NaN,3,31.0,healthy for them yogurt popsicles,2003-07-26,15-minutes-or-less;time-to-make;course;prepara...
2,1533,"these were so go, it surprised even me.",38798,30,8.0,5,17.0,i can t believe it s spinach,2002-08-29,30-minutes-or-less;time-to-make;course;main-in...
3,22724,my sister-in-law made these for us at a family...,35173,45,NaN,7,11.0,italian gut busters,2002-07-27,60-minutes-or-less;time-to-make;course;prepara...
4,4470,i think a fondue is a very romantic casual din...,84797,25,NaN,4,19.0,love is in the air beef fondue sauces,2004-02-23,30-minutes-or-less;time-to-make;course;main-in...


1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

In [272]:
import csv
dict_ingredients = {}
with open('data/ingredients_sample.csv') as fp:
    reader = csv.DictReader(fp)
    for row in reader:
        dict_ingredients.setdefault(row['recipe_id'],[]).append(row['ingredient'])
dict_ingredients['44123']

['unsalted butter',
 'carrot',
 'onion',
 'celery',
 'broccoli stem',
 'dried thyme',
 'dried oregano',
 'dried sweet basil leaves',
 'dry white wine',
 'chicken stock',
 'worcestershire sauce',
 'tabasco sauce',
 'smoked chicken',
 'black beans',
 'broccoli floret',
 'heavy cream',
 'salt & fresh ground pepper',
 'cornstarch']

1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

In [50]:
dict_ingredients_str = {}
len_ingredients = []
for i in dict_ingredients:
    dict_ingredients_str[i] = '*'.join(dict_ingredients[i])
    len_ingredients.append(len(dict_ingredients[i]))
df_ingredients = pd.DataFrame.from_dict(dict_ingredients_str,orient = 'index').reset_index().rename(columns={0:'ingredients','index':'id'})
df_ingredients['n_ingredients'] = len_ingredients
recipes = recipes.combine_first(df_ingredients)
recipes.head(5)

,contributor_id,description,id,ingredients,minutes,n_ingredients,n_steps,n_tags,name,submitted,tags
0,35193,an original recipe created by chef scott meska...,44123,unsalted butter*carrot*onion*celery*broccoli s...,90,18.0,11,25.0,george s at the cove black bean soup,2002-10-25,weeknight;time-to-make;course;main-ingredient;...
1,91970,my children and their friends ask for my homem...,67664,unsalted butter*all-purpose flour*walnuts*ligh...,10,6.0,3,31.0,healthy for them yogurt popsicles,2003-07-26,15-minutes-or-less;time-to-make;course;prepara...
2,1533,"these were so go, it surprised even me.",38798,unsalted butter*onion*milk*salt*egg*cream chee...,30,8.0,5,17.0,i can t believe it s spinach,2002-08-29,30-minutes-or-less;time-to-make;course;main-in...
3,22724,my sister-in-law made these for us at a family...,35173,unsalted butter*milk*eggs*honey*white bread*va...,45,8.0,7,11.0,italian gut busters,2002-07-27,60-minutes-or-less;time-to-make;course;prepara...
4,4470,i think a fondue is a very romantic casual din...,84797,unsalted butter*nuts*granulated sugar*semi-swe...,25,4.0,4,19.0,love is in the air beef fondue sauces,2004-02-23,30-minutes-or-less;time-to-make;course;main-in...


1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

In [58]:
sum(recipes['n_ingredients'].isna())

0

In [57]:
recipes['n_ingredients'] = recipes['n_ingredients'].astype(int)
recipes.to_csv('data/recipes_sample_with_tags_ingredients.csv', index=False) 

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

In [68]:
import numpy as np
arr_lower_2000 = recipes[recipes['submitted'].dt.year<2000][['contributor_id','id','minutes','n_ingredients','n_steps','n_tags']].to_numpy()
arr_higher_2000 = recipes[recipes['submitted'].dt.year>=2000][['contributor_id','id','minutes','n_ingredients','n_steps','n_tags']].to_numpy()

In [86]:
arr_lower_2000

array([[1.5620e+03, 3.4410e+03, 3.0000e+01, 8.0000e+00, 8.0000e+00,
        1.0000e+01],
       [1.6170e+03, 4.2050e+03, 2.5000e+01, 5.0000e+00, 3.0000e+00,
        1.4000e+01],
       [1.5340e+03, 3.2580e+03, 0.0000e+00, 6.0000e+00, 8.0000e+00,
        2.0000e+01],
       ...,
       [1.5350e+03, 3.7520e+03, 0.0000e+00, 4.0000e+00, 1.3000e+01,
        9.0000e+00],
       [1.5980e+03, 4.8010e+03, 2.0000e+01, 7.0000e+00, 4.0000e+00,
        1.8000e+01],
       [1.2403e+05, 2.9820e+03, 0.0000e+00, 6.0000e+00, 6.0000e+00,
        1.3000e+01]])

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

In [81]:
np.savez('data/arr.npz',arr_lower_2000=arr_lower_2000,arr_higher_2000=arr_higher_2000)

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

In [92]:
r = np.load('data/arr.npz')
r['arr_lower_2000']

array([[1.5620e+03, 3.4410e+03, 3.0000e+01, 8.0000e+00, 8.0000e+00,
        1.0000e+01],
       [1.6170e+03, 4.2050e+03, 2.5000e+01, 5.0000e+00, 3.0000e+00,
        1.4000e+01],
       [1.5340e+03, 3.2580e+03, 0.0000e+00, 6.0000e+00, 8.0000e+00,
        2.0000e+01],
       ...,
       [1.5350e+03, 3.7520e+03, 0.0000e+00, 4.0000e+00, 1.3000e+01,
        9.0000e+00],
       [1.5980e+03, 4.8010e+03, 2.0000e+01, 7.0000e+00, 4.0000e+00,
        1.8000e+01],
       [1.2403e+05, 2.9820e+03, 0.0000e+00, 6.0000e+00, 6.0000e+00,
        1.3000e+01]])

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

In [245]:
import h5py
datasets = []
names = []
attrs = []
with h5py.File('data/nutrition_sample.h5', 'r') as hdf:
    for dataset in hdf:
        datasets.append(hdf[dataset][:])
        names.append(str(dataset))
        attrs.append(list(hdf[dataset].attrs.values()))
        print('Dataset name=' + str(dataset) + ', dataset size=' + str(hdf[dataset].shape) + ', metadata=' + str(list(hdf[dataset].attrs.values())))

Dataset name=dataset_0, dataset size=(30000, 2), metadata=['recipe_id', 'calories (#)']
Dataset name=dataset_1, dataset size=(30000, 2), metadata=['recipe_id', 'total fat (PDV)']
Dataset name=dataset_2, dataset size=(30000, 2), metadata=['recipe_id', 'sugar (PDV)']
Dataset name=dataset_3, dataset size=(30000, 2), metadata=['recipe_id', 'sodium (PDV)']
Dataset name=dataset_4, dataset size=(30000, 2), metadata=['recipe_id', 'protein (PDV)']
Dataset name=dataset_5, dataset size=(30000, 2), metadata=['recipe_id', 'saturated fat (PDV)']
Dataset name=dataset_6, dataset size=(30000, 2), metadata=['recipe_id', 'carbohydrates (PDV)']


['recipe_id', 'calories (#)']

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

In [248]:
datasets_lower_100 = []
datasets_higher_100 = []
for dataset in datasets:
    datasets_higher_100.append(dataset[dataset[:,1]>100])
    datasets_lower_100.append(dataset[dataset[:,1]<=100])
with h5py.File('data/nutrition_grouped.h5', 'w') as f:
    lower_100 = f.create_group('lower_100')
    higher_100 = f.create_group('higher_100')
    for i in range(len(names)):
        low_100 = lower_100.create_dataset(names[i], data=datasets_lower_100[i])
        high_100 = higher_100.create_dataset(names[i], data=datasets_higher_100[i])
        low_100.attrs['col_0'] = attrs[i][0]
        low_100.attrs['col_1'] = attrs[i][1]
        high_100.attrs['col_0'] = attrs[i][0]
        high_100.attrs['col_1'] = attrs[i][1]g


3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

In [265]:
with h5py.File('data/nutrition_grouped.h5', 'r') as hdf:
    for group in hdf:
        for dataset in hdf[group]:
            print('group=' + str(group) + ', Dataset name=' + str(dataset) + ', dataset size=' + str(hdf[group][dataset].shape) + ', metadata=' + str(list(hdf[group][dataset].attrs.values())))

group=higher_100, Dataset name=dataset_0, dataset size=(26733, 2), metadata=['recipe_id', 'calories (#)']
group=higher_100, Dataset name=dataset_1, dataset size=(1736, 2), metadata=['recipe_id', 'total fat (PDV)']
group=higher_100, Dataset name=dataset_2, dataset size=(5316, 2), metadata=['recipe_id', 'sugar (PDV)']
group=higher_100, Dataset name=dataset_3, dataset size=(1244, 2), metadata=['recipe_id', 'sodium (PDV)']
group=higher_100, Dataset name=dataset_4, dataset size=(1776, 2), metadata=['recipe_id', 'protein (PDV)']
group=higher_100, Dataset name=dataset_5, dataset size=(2858, 2), metadata=['recipe_id', 'saturated fat (PDV)']
group=higher_100, Dataset name=dataset_6, dataset size=(642, 2), metadata=['recipe_id', 'carbohydrates (PDV)']
group=lower_100, Dataset name=dataset_0, dataset size=(3267, 2), metadata=['recipe_id', 'calories (#)']
group=lower_100, Dataset name=dataset_1, dataset size=(28264, 2), metadata=['recipe_id', 'total fat (PDV)']
group=lower_100, Dataset name=datase

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.

In [266]:
with h5py.File('data/nutrition_grouped_compressed.h5', 'w') as f:
    lower_100 = f.create_group('lower_100')
    higher_100 = f.create_group('higher_100')
    for i in range(len(names)):
        low_100 = lower_100.create_dataset(names[i], data=datasets_lower_100[i],compression="gzip")
        high_100 = higher_100.create_dataset(names[i], data=datasets_higher_100[i],compression="gzip")
        low_100.attrs['col_0'] = attrs[i][0]
        low_100.attrs['col_1'] = attrs[i][1]
        high_100.attrs['col_0'] = attrs[i][0]
        high_100.attrs['col_1'] = attrs[i][1]

In [267]:
#Без сжатия 3290кб
#Со сжатием 1158кб -> очевидно объем меньше